In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm,tqdm_notebook
import glob


In [ ]:
NOMAD2_result_folder=''

In [ ]:
### read in p-values, afetr_correction.scores.csv file
fname=NOMAD2_result_folder+'/result.after_correction.scores.tsv'
df = pd.read_csv(fname,sep='\t')

In [ ]:
#### can change thresholds

effectSizeThresh = np.quantile((df[
    (df.pval_opt_corrected<.05) &
                (df.M>1000)]).effect_size_bin,[.9])[0]
print('effect size thresh', effectSizeThresh)

df_filtered = (df[(df.pval_opt_corrected<.05) &
                (df.effect_size_bin>effectSizeThresh) &
                (df.M>1000)])

reducedAnchLst = df_filtered.anchor.unique()
print(df.anchor.nunique(),len(reducedAnchLst))

In [ ]:
### load in c vectors, keeping only the ones corresponding to relevant anchors
### this is currently fast enough, but for larger datasets may need faster methods here, utilizing SATC tools
dfArr = []

##### path to intermediary_files output folder
fldrName= NOMAD2_result_folder+'/result_Cjs/'

for fname in tqdm(glob.glob(fldrName+'bin*.cjs')):
    dfArr.append(
        pd.concat(chunk[chunk.anchor.isin(reducedAnchLst)] for chunk in pd.read_csv(fname,sep='\t',chunksize=1E7))
    )
    
## concatenate
dfcj = pd.concat(dfArr)

### pivot to matrix form
pivotedDf = dfcj.pivot(index=['anchor'], columns='sample', values='Cj').fillna(0)